In [3]:
import pandas as pd

## Cleaning scrapped Data of Coop stores 
* first take a look at the data for supermarket
* then all stores with coop in the name
* output csv:
    - all coop store counted for each plz
    - separated plz count for each store with coop in the name 
        - listed in a dataframe [shopname, plz, count]
        - pivot in a dataframe [storename,plz]

#### Take a look at the data (only supermarket)

In [6]:
with open("../../data/raw_data/coop_standorte_ch_supermarket.json") as json_file:
    json_coop = json.load(json_file)
type(json_coop)

dict

In [7]:
json_coop.keys()

dict_keys(['vstList'])

In [8]:
normelaize_coop=pd.json_normalize(json_coop['vstList'])
#normelaize_coop

In [9]:
normelaize_coop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   namePublic                  714 non-null    object 
 1   orgVlcRegNr                 714 non-null    object 
 2   logoAltText                 714 non-null    object 
 3   distance                    714 non-null    float64
 4   strasse                     714 non-null    object 
 5   language                    714 non-null    object 
 6   logoSmall                   714 non-null    object 
 7   ort                         714 non-null    object 
 8   nameInternal                714 non-null    object 
 9   formatId                    714 non-null    object 
 10  prettyUrlName               714 non-null    object 
 11  name                        714 non-null    object 
 12  hausnummer                  714 non-null    object 
 13  logo                        714 non

In [10]:
df_coop=pd.DataFrame(json_coop['vstList'])

In [11]:
df_coop.head(2)

,namePublic,orgVlcRegNr,logoAltText,distance,strasse,betriebsNummerId,language,logoSmall,ort,nameInternal,...,name,hausnummer,logo,openingHours,geoKoordinaten,postalRegion,orgVlRegNr,orgObId,plz,myRegion
0,Coop Supermarkt Giswil,NWZZ-06,Coop Supermarkt - Logo,3820.913043,BrÃ¼nigstrasse,{'id': '4956'},DE,https://libs.coop.ch/logos-mini-vst/coop_retai...,Giswil,Coop Supermarkt Giswil 4956,...,Giswil,42,https://libs.coop.ch/logos-vst/coop-retail-de.svg,"[{'time': '07:00 - 18:00', 'day': 'SATURDAY', ...","{'xCoordinate': 657026, 'yCoordinate': 187653,...",{'region': 'ZZ'},-,VRENWZZ,6074,ZZ
1,Coop Supermarkt Sarnen,NWZZ-06,Coop Supermarkt - Logo,8979.498834,Marktstrasse,{'id': '1969'},DE,https://libs.coop.ch/logos-mini-vst/coop_retai...,Sarnen,Coop Supermarkt Sarnen 1969,...,Sarnen,32,https://libs.coop.ch/logos-vst/coop-retail-de.svg,"[{'time': '07:30 - 18:00', 'day': 'SATURDAY', ...","{'xCoordinate': 661919, 'yCoordinate': 194342,...",{'region': 'ZZ'},-,VRENWZZ,6060,ZZ


In [12]:
# How much plz regions
df_coop['plz'].nunique()

534

In [13]:
#df_coop['plz'].unique()

In [14]:
# Count the supermarkt per plz
df_coop['plz'].value_counts()

plz
3011    8
4058    7
8050    6
8048    6
3600    5
       ..
8965    1
3048    1
3312    1
8802    1
1933    1
Name: count, Length: 534, dtype: int64

In [15]:
# Use the column 'logoAltText', while the aren't city names or other noises
df_coop['logoAltText'].value_counts()

logoAltText
Coop Supermarkt - Logo    714
Name: count, dtype: int64

In [16]:
plz_count=df_coop.groupby(by='plz')['logoAltText'].value_counts()

In [17]:
plz_count

plz   logoAltText           
1634  Coop Supermarkt - Logo    1
1636  Coop Supermarkt - Logo    1
1637  Coop Supermarkt - Logo    1
1660  Coop Supermarkt - Logo    1
1700  Coop Supermarkt - Logo    4
                               ..
9548  Coop Supermarkt - Logo    1
9630  Coop Supermarkt - Logo    1
9642  Coop Supermarkt - Logo    1
9650  Coop Supermarkt - Logo    1
9657  Coop Supermarkt - Logo    1
Name: count, Length: 534, dtype: int64

#### Use data for all stores 

In [19]:
# read for all coop stores

with open("../../data/raw_data/coop_standorte_ch_p.json") as json_file:
    json_coop_p = json.load(json_file)
df_coop_all=pd.DataFrame(json_coop_p['vstList'])

In [68]:
# Uniques at the column 'logoAltText' are usefull to filter the Coop shops of intest
names_stores=df_coop_all['logoAltText'].unique()
store_count=df_coop_all['logoAltText'].value_counts()

In [90]:
# choose the coops stores of interest
list_coop=[]
count_list=[]
for i,stores in enumerate(names_stores):
    if 'Coop' in stores:
        list_coop.append(stores)
        count_list.append([stores.replace(' - Logo',''),store_count.iloc[i]])

In [100]:
# DataFrame for the count of each Coop Store
df_coopstores_count=pd.DataFrame(count_list, columns=['Storename', 'count'])[:-1] # drops 'Coop Tagungzentrun'
df_coopstores_count.to_csv('../../data/cleaned_data/df_coopstores_count.csv', index=True)
df_coopstores_count

,Storename,count
0,Coop Supermarkt,959
1,Coop to go,327
2,Coop Vitality,63
3,Coop City,30
4,Coop Bistro,23
5,Coop Pronto,11


In [22]:
# create a dataframe with storesname and plz
plz_list=[]
for i,stores in enumerate(df_coop_all['logoAltText']):
    if stores in list_coop:
        store_name=stores.replace(' - Logo','')
        plz_list.append([df_coop_all['plz'][i],store_name])
        
df_plz_list=pd.DataFrame(plz_list,columns=['zip', 'stores'])
df_plz_list.head()

,zip,stores
0,8001,Coop Supermarkt
1,8001,Coop to go
2,8021,Coop Supermarkt
3,8001,Coop Vitality
4,8001,Coop Supermarkt


In [23]:
# groupby the stores per plz
stores_count_byplz=df_plz_list.groupby(by='zip').value_counts()
type(stores_count_byplz)

pandas.core.series.Series

In [66]:
df_plz_list.groupby(by='stores').value_counts()['Coop Bistro'].sum()
#stores_count

158

In [71]:
# reset the index and sum for each plz
stores_count_byplz=stores_count_byplz.reset_index()
allcoop_plz=stores_count_byplz.groupby('zip').sum('count')
df_plz_allstores=allcoop_plz.reset_index()
df_plz_allstores.head()

,zip,count
0,1002,1
1,1003,8
2,1004,3
3,1005,1
4,1006,3


#### check the results 
- raw data
- cleaned data
- on the website


In [73]:
#check sum of the stores 
#checked also on the website
df_plz_allstores['count'].sum()

1585

In [75]:
df_plz_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1585 entries, 0 to 1584
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   zip     1585 non-null   object
 1   stores  1585 non-null   object
dtypes: object(2)
memory usage: 24.9+ KB


### CSV for sum of all stores (with coop in the name into)

In [123]:
df_plz_allstores.to_csv('../../data/cleaned_data/df_allcoop_zipcount.csv', index=False)

### CSV for each Coop form 

- Coop Supermarkt
- Coop City
- Coop to go
- Coop Bistro
- Coop Vitality
- Coop Pronto
- Coop Tagungszentrum


In [115]:
df_differtent_stores_count=pd.DataFrame(df_plz_list.groupby(by=['stores'])['zip'].value_counts())
df_diff_coops_list=df_differtent_stores_count.reset_index()
df_diff_coops_list.to_csv('../../data/cleaned_data/df_coopstores_zipcount_list.csv', index=False)

In [132]:
df_diff_coops=df_differtent_stores_count.unstack()
df_diff_coops=df_diff_coops.fillna(0)
df_diff_coops.to_csv('../../data/cleaned_data/df_coopstores_zipcount.csv', index=True)

In [134]:
df_diff_coops

count                                               ...  \
zip                  1002 1003 1004 1005 1006 1007 1008 1009 1010 1012  ...   
stores                                                                  ...   
Coop Bistro           1.0  2.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...   
Coop City             0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
Coop Pronto           0.0  2.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...   
Coop Supermarkt       0.0  2.0  2.0  0.0  2.0  2.0  3.0  1.0  1.0  1.0  ...   
Coop Tagungszentrum   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
Coop Vitality         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
Coop to go            0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   

                                                                       
zip                 9500 9524 9532 9542 9545 9548 9630 9642 9650 9657  
stores                                                                 
Coop Bistro          1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
Coop City            1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
Coop Pronto          2.0  0.0  0.0  1.0  0.0  1.0  1.0  0.0  0.0  0.0  
Coop Supermarkt      1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
Coop Tagungszentrum  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
Coop Vitality        0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
Coop to go           0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[7 rows x 768 columns]

### CSV for Coop Proto

In [121]:
df_Coop_Proto=df_diff_coops_list[df_diff_coops_list['stores']=='Coop Pronto']
df_Coop_Proto.to_csv('../../data/cleaned_data/df_coop_proto_zipcount.csv', index=False)

In [ ]:
COOP_stores['zip']=COOP_stores['zip'].astype(str)
type(COOP_stores['zip'][1])